# Notebook on Google Colab Data

In [2]:
! pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=ae26c3e3043d84cf137dbe6bfc656bc5e893937e3c1a330c60018abec3f6014e
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [3]:
from pyspark.sql import SparkSession

In [5]:
spark = SparkSession.builder.master("local").appName("ScenarioBasedQtns").getOrCreate()

In [6]:
sc = spark.sparkContext

In [11]:
rdd_in = sc.textFile("/content/titanic.csv")

In [12]:
rdd_in.take(4)

['PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked',
 '1,0,3,"Braund, Mr. Owen Harris",male,22,1,0,A/5 21171,7.25,,S',
 '2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Thayer)",female,38,1,0,PC 17599,71.2833,C85,C',
 '3,1,3,"Heikkinen, Miss. Laina",female,26,0,0,STON/O2. 3101282,7.925,,S']

# using rdds

In [43]:
#rdd_data=rdd_in.filter(lambda x: not x.startswith("PassengerId"))
#rdd_header=rdd_in.filter(lambda x: x.startswith("PassengerId"))
rdd_header=rdd_in.first()
print(rdd_header)

data_rdd=rdd_in.filter(lambda x: x != rdd_header)
data_rdd.take(4)

PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked


['1,0,3,"Braund, Mr. Owen Harris",male,22,1,0,A/5 21171,7.25,,S',
 '2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Thayer)",female,38,1,0,PC 17599,71.2833,C85,C',
 '3,1,3,"Heikkinen, Miss. Laina",female,26,0,0,STON/O2. 3101282,7.925,,S',
 '4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35,1,0,113803,53.1,C123,S']

In [46]:
split_rdd = data_rdd.map(lambda row: row.split(",")).flatMap(lambda row: [(row[0], name_part) for name_part in row[3].split(" ")])
split_rdd.take(5)

[('1', '"Braund'),
 ('2', '"Cumings'),
 ('3', '"Heikkinen'),
 ('4', '"Futrelle'),
 ('5', '"Allen')]

# Using DFs

In [49]:
df = spark.read.csv("/content/titanic.csv", inferSchema=True, header=True)

df.show(5, False)

+-----------+--------+------+---------------------------------------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|Name                                               |Sex   |Age |SibSp|Parch|Ticket          |Fare   |Cabin|Embarked|
+-----------+--------+------+---------------------------------------------------+------+----+-----+-----+----------------+-------+-----+--------+
|1          |0       |3     |Braund, Mr. Owen Harris                            |male  |22.0|1    |0    |A/5 21171       |7.25   |NULL |S       |
|2          |1       |1     |Cumings, Mrs. John Bradley (Florence Briggs Thayer)|female|38.0|1    |0    |PC 17599        |71.2833|C85  |C       |
|3          |1       |3     |Heikkinen, Miss. Laina                             |female|26.0|0    |0    |STON/O2. 3101282|7.925  |NULL |S       |
|4          |1       |1     |Futrelle, Mrs. Jacques Heath (Lily May Peel)       |female|35.0|1    |0    |113803          |53

In [55]:
df.select("PassengerId", "Name", "Ticket").show(5, False)

+-----------+---------------------------------------------------+----------------+
|PassengerId|Name                                               |Ticket          |
+-----------+---------------------------------------------------+----------------+
|1          |Braund, Mr. Owen Harris                            |A/5 21171       |
|2          |Cumings, Mrs. John Bradley (Florence Briggs Thayer)|PC 17599        |
|3          |Heikkinen, Miss. Laina                             |STON/O2. 3101282|
|4          |Futrelle, Mrs. Jacques Heath (Lily May Peel)       |113803          |
|5          |Allen, Mr. William Henry                           |373450          |
+-----------+---------------------------------------------------+----------------+
only showing top 5 rows



In [56]:
df.select("PassengerId", "Name", "Ticket").coalesce(1).write.csv("/content/titanicop.csv",header=True)